In [21]:
import os
import shutil

def copy_and_rename_images(src_folder, dst_folder):
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    
    for root, dirs, files in os.walk(src_folder):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
                src_path = os.path.join(root, filename)
                
                # Extract the immediate parent folder name and the grandparent folder name
                parent_folder = os.path.basename(os.path.dirname(src_path))
                grandparent_folder = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(src_path))))
                
                # Create the destination folder structure
                relative_path = os.path.relpath(root, src_folder)
                new_dst_folder = os.path.join(dst_folder, relative_path)
                if not os.path.exists(new_dst_folder):
                    os.makedirs(new_dst_folder)
                
                # Construct the new filename
                dst_filename = f"{grandparent_folder}_{filename}"
                dst_path = os.path.join(new_dst_folder, dst_filename)
                
                # Copy and rename the file
                # shutil.copy2(src_path, dst_path)
                print(f"Copied and renamed: {src_path} to {dst_path}")

# Example usage:
src_folder = '/Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry/03_11_2021/Ground_RGB_Photos'
dst_folder = '/Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry v2/Ground_RGB_Photos/All'

copy_and_rename_images(src_folder, dst_folder)

Copied and renamed: /Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry/03_11_2021/Ground_RGB_Photos/Healthy/11-7.jpg to /Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry v2/Ground_RGB_Photos/All/Healthy/03_11_2021_11-7.jpg
Copied and renamed: /Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry/03_11_2021/Ground_RGB_Photos/Healthy/8-13.jpg to /Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry v2/Ground_RGB_Photos/All/Healthy/03_11_2021_8-13.jpg
Copied and renamed: /Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry/03_11_2021/Ground_RGB_Photos/Healthy/6-16.jpg to /Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry v2/Ground_RGB_Photos/All/Healthy/03_11_2021_6-16.jpg
Copied and renamed: /Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry/03_11_2021/Ground_RGB_Photos/Healthy/10-51.jpg to /Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry v2/Ground_RGB_Photos/All/Healthy/03_11_2021_10-51.jpg
Copied and ren

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2

In [ ]:
def process_image(image_path, output_path):
    # Load the image
    image = Image.open(image_path)

    # Convert the image to an array for processing
    image_array = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Convert the image to grayscale for leaf detection
    gray_image = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)

    # Use GaussianBlur to reduce noise and improve edge detection
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Use Canny edge detection with different thresholds
    edges = cv2.Canny(blurred_image, threshold1=30, threshold2=100)

    # Find contours in the edged image
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours by area to remove noise and focus on larger objects
    min_contour_area = 100  # Minimum contour area to keep
    filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > min_contour_area]

    # Draw bounding boxes around the filtered contours
    image_with_filtered_boxes = image_array.copy()
    for contour in filtered_contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image_with_filtered_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Save the image with filtered bounding boxes
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    result_image = Image.fromarray(cv2.cvtColor(image_with_filtered_boxes, cv2.COLOR_BGR2RGB))
    result_image.save(output_path)

In [ ]:
def process_directory(input_dir, output_dir):
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_path = os.path.join(root, file)
                output_path = os.path.join(output_dir, os.path.relpath(input_path, input_dir))
                process_image(input_path, output_path)

In [ ]:
# Define the input and output directories
input_directory = "/Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry v2/Ground_RGB_Photos/All"
output_directory = "/Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry v2/Ground_RGB_Photos/Bounded"

# Process all images in the input directory
process_directory(input_directory, output_directory)

In [ ]:
import os
import cv2
from ultralytics import YOLO
import numpy as np
from PIL import Image

def process_image_with_yolo(image_path, output_path, model):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}")
        return

    # Perform inference with YOLOv8
    results = model(image)

    # Draw bounding boxes on the image
    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Save the image with bounding boxes
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, image)

def process_directory_with_yolo(input_dir, output_dir, model):
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_path = os.path.join(root, file)
                output_path = os.path.join(output_dir, os.path.relpath(input_path, input_dir))
                process_image_with_yolo(input_path, output_path, model)

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # Load the pretrained YOLOv8 nano model

# Define the input and output directories
input_directory = "/Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry v2/Ground_RGB_Photos/All"
output_directory = "/Users/izzymohamed/Desktop/Vision For Social Good/DATA/Cherry v2/Ground_RGB_Photos/Bounded2"

# Process all images in the input directory
process_directory_with_yolo(input_directory, output_directory, model)